In [32]:
class Rule():
    def __init__(self, field, operator, workflow) -> None:
        self.field = field
        self.operator = operator
        self.workflow = workflow

    def apply_rule(self, range):
        limited_range, remainder = self.operator(range, self.field)
        return limited_range, self.workflow, remainder 

In [33]:
class Workflow():
    def __init__(self, name, rules) -> None:
        self.rules = rules
        self.name = name

    def apply_rules(self, range):
        new_ranges = []
        for rule in self.rules:
            limited_range, flow_name, remainder  = rule.apply_rule(range)
            if limited_range:
                new_ranges.append((limited_range, flow_name ))
            if remainder:
                range = remainder
            else:
                break
        return new_ranges


In [34]:
class Workflows():
    def __init__(self, workflows) -> None:
        self.workflows = workflows

    def compute_accepted(self, range):
        workflow_name = "in"
        ranges = [(range, workflow_name)]
        accepted = []
        while True:
            to_pass = [range for range in ranges if range[1] not in ["A", 'R']]
            accepted += [range for range in ranges if range[1] == 'A']
            if not len(to_pass):
                break
            new_ranges = []
            for new in to_pass:
                wf_name = new[1]
                new_ranges += self.workflows[wf_name].apply_rules(new[0])
            ranges = new_ranges
        return accepted


In [35]:
class XMASRange():
    def __init__(self, x, m, a, s) -> None:
        self.x = x
        self.m = m
        self.a = a
        self.s = s

    def __repr__(self) -> str:
        return f"{self.x=} {self.m=} {self.a=} {self.s=}"
    
    def compute_possibilities(self):
        x = self.x[1] - self.x[0] + 1
        m = self.m[1] - self.m[0] + 1
        a = self.a[1] - self.a[0] + 1
        s = self.s[1] - self.s[0] + 1
        return x*m*a*s

In [36]:
def split_range_less(range: XMASRange, field_name, value):
    current_from, current_to = getattr(range, field_name)
    new_to = min(value-1, current_to)
    if new_to < current_from:
        return None, range
    val_map = {
        "x": range.x,
        "m": range.m,
        "a": range.a,
        "s": range.s
    }
    val_map[field_name] = (current_from, new_to)
    range_left = XMASRange(val_map["x"], val_map["m"],val_map["a"],val_map["s"])
    val_map[field_name] = (new_to+1, current_to)
    range_right = XMASRange(val_map["x"], val_map["m"],val_map["a"],val_map["s"])
    return range_left, range_right if new_to+1 <= current_to else None
    


In [37]:
split_range_less(XMASRange((1,4000),(1,4000),(1,4000),(1,4000)), "x", 1000)

(self.x=(1, 999) self.m=(1, 4000) self.a=(1, 4000) self.s=(1, 4000),
 self.x=(1000, 4000) self.m=(1, 4000) self.a=(1, 4000) self.s=(1, 4000))

In [38]:
def split_range_greater(range: XMASRange, field_name, value):
    current_from, current_to = getattr(range, field_name)
    new_from = max(value+1, current_from)
    if current_to < new_from:
        return None, range
    val_map = {
        "x": range.x,
        "m": range.m,
        "a": range.a,
        "s": range.s
    }
    val_map[field_name] = (new_from, current_to)
    range_left = XMASRange(val_map["x"], val_map["m"],val_map["a"],val_map["s"])
    val_map[field_name] = (current_from, new_from-1)
    range_right = XMASRange(val_map["x"], val_map["m"],val_map["a"],val_map["s"])
    return range_left, range_right if current_from <= new_from-1 else None

In [39]:
split_range_greater(XMASRange((1,4000),(1,4000),(1,4000),(1,4000)), "x", 1000)

(self.x=(1001, 4000) self.m=(1, 4000) self.a=(1, 4000) self.s=(1, 4000),
 self.x=(1, 1000) self.m=(1, 4000) self.a=(1, 4000) self.s=(1, 4000))

In [40]:
split_range_greater(XMASRange((1,4000),(1,4000),(1,4000),(1,4000)), "x", 0)

(self.x=(1, 4000) self.m=(1, 4000) self.a=(1, 4000) self.s=(1, 4000), None)

In [41]:
def copy_range(range: XMASRange, field_name, value):
    return XMASRange(range.x, range.m,range.a,range.s)

In [42]:
def copy_range_wo_rem(range: XMASRange):
    return XMASRange(range.x, range.m,range.a,range.s), None

In [43]:
def parse_instruction(instruction):
    if "<" in instruction:
        field, value = instruction.split("<")
        value, flow = value.split(":")
        value = int(value)
        return Rule(field, (lambda range, field_name: split_range_less(range, field_name, value)), flow)
    if ">" in instruction:
        field, value = instruction.split(">")
        value, flow = value.split(":")
        value = int(value)
        return Rule(field, (lambda range, field_name: split_range_greater(range, field_name, value)), flow)
    return Rule("x", (lambda range, field_name: copy_range_wo_rem(range)), instruction)

In [44]:
parsing_items = False
with open("day19.txt") as f:
    lines = f.readlines()
instructions = []
workflows = {}
items = []
for i, line in enumerate(lines):
    if len(line) > 3:
        if i != len(lines) -1:
            line = line[:-1]
        if parsing_items:
            values = line[1:-1]
            values = values.split(',')
            val_map = {}
            for value in values:
                name, v = value.split("=")
                val_map[name] = int(v)
            #items.append(XMAS(val_map["x"], val_map["m"],val_map["a"],val_map["s"]))
        else:
        
            name, instructions = line.split("{")
            instructions = instructions[:-1]
            #print(f"{name=} {instructions=}")
            instructions = instructions.split(",")
            rules = [parse_instruction(instruction) for instruction in instructions]
            workflow = Workflow(name, rules)
            workflows[name] = workflow

    else:
        parsing_items = True
workflows = Workflows(workflows)

In [45]:
#items

In [46]:
init_range = XMASRange((1,4000),(1,4000),(1,4000),(1,4000))
accepted = workflows.compute_accepted(init_range)

In [47]:
total = 0
for range, state in accepted:
    total += range.compute_possibilities()
total

116606738659695